In [130]:
import json
import pandas as pd
import string
import spacy

In [131]:
df = pd.read_csv(r"C:\Users\SNIGGDHA\Downloads\zomato.csv\zomato.csv")

In [133]:
df['Rest_id'] = range (1,len(df)+1)
df_finals = pd.DataFrame()

In [134]:
filtered_df = df[(df['Rest_id'] >= 1) & (df['Rest_id'] <= 100)]

url = filtered_df['url'].tolist()

In [135]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

Header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
}

#url = [
    #'https://www.zomato.com/bangalore/rosewood-international-hotel-bar-restaurant-banashankari-bangalore?',
 #   'https://www.zomato.com/bangalore/rosewood-international-hotel-bar-restaurant-mysore-road-bangalore?context=eyJzZSI6eyJlIjpbIjU5MDkwIiwiMTgzODI5NDQiLCIxODIyNDY3NiIsIjU5Mjg5IiwxODM3MzM4Nl0sInQiOiJSZXN0YXVyYW50cyBpbiBCYW5hc2hhbmthcmkgc2VydmluZyBCdWZmZXQifX0='
#]

for i in url:
    original_url = i
    Rest_Name = df[df['url'] == original_url]['name']
    Rest_id = df[df['url'] == original_url]['Rest_id'].iloc[0]
    #top_restid = df['restid'].iloc[0]
    #print("Top RestID:", top_restid)
    end_index = i.find("?")

    if end_index != -1:
        base_url = i[:end_index] + "/order"
    else:
        base_url = i

    response = requests.get(base_url, headers=Header)
    print(base_url, ';', response.status_code)

    soup_per_page = bs(response.content, 'html.parser')

    container = soup_per_page.find_all("section")
    
    for section in container:
        Course_type=[]
        Menu = []
        Menu_Type = []
        BestSeller = []
        Votes = []
        Dish_Rating = []
        h4_tags_course = section.find('h4', class_=lambda x: x and 'sc-1hp8d8a-0' in x.split())
        if h4_tags_course is None:
            continue
        else:
            h4_tags_text = h4_tags_course.text.strip()
            if h4_tags_text:
                h4_tags_text_rep = h4_tags_text.replace("[", "").replace("]", "")
                Course_type.append(h4_tags_text_rep)

        container_menu = section.find_all("div", class_=False, attrs=False)

        if len(container_menu) == 0:
            continue
        else:
            for div in container_menu:
                h4_tags = div.find("h4", class_="sc-1s0saks-15 iSmBPS")
                if h4_tags:
                    for h4 in h4_tags:
                        Menu.append(h4)
                else:
                    Menu.append("NA")

                type_tag = div.find('div', attrs={'type': True})
                if type_tag:
                    type_value = type_tag['type']
                    Menu_Type.append(type_value)
                else:
                    Menu_Type.append("NA")

                if div and 'BESTSELLER' in div.text:
                    BestSeller.append('BestSeller')
                else:
                    BestSeller.append('Not BestSeller')

                def find_votes_div(tag):
                    return tag.name == 'span' and 'votes' in tag.text

                votes_div = div.find(find_votes_div)
                if votes_div:
                    votes_text = votes_div.text.strip()
                    Votes.append(votes_text)
                else:
                    votes_text = 0
                    Votes.append(votes_text)

                div_tag = div.find('div', class_='sc-z30xqq-3 bewuUV')
                count = 0
                if div_tag:
                    i_tags = div_tag.find_all('i', color='#F3C117')
                    for i in i_tags:
                        if i.find('title') and i.find('title').text == 'star-fill':
                            count += 1
                        else:
                            count += 0.5
                    Dish_Rating.append(count)
                else:
                    count += 0
                    Dish_Rating.append(count)

        df_iteration = pd.DataFrame({
            'Rest_id' : Rest_id * len(Menu),
            'URL': base_url * len(Menu),
            'Rest_Name': [Rest_Name] * len(Menu),
            'Course_Type' : Course_type * len(Menu),
            'menu': Menu,
            'Menu Type': Menu_Type,
            'BestSeller': BestSeller,
            'DishVotes': Votes,
            'DishRating': Dish_Rating
        })

        df_finals = pd.concat([df_finals, df_iteration], ignore_index=True) 

print(df_finals)


https://www.zomato.com/bangalore/jalsa-banashankari/order ; 200
https://www.zomato.com/bangalore/spice-elephant-banashankari/order ; 200
https://www.zomato.com/SanchurroBangalore/order ; 200
https://www.zomato.com/bangalore/addhuri-udupi-bhojana-banashankari/order ; 200
https://www.zomato.com/bangalore/grand-village-basavanagudi/order ; 200
https://www.zomato.com/bangalore/timepass-dinner-basavanagudi/order ; 200
https://www.zomato.com/bangalore/rosewood-international-hotel-bar-restaurant-mysore-road-bangalore/order ; 200
https://www.zomato.com/bangalore/onesta-banashankari/order ; 200
https://www.zomato.com/bangalore/penthouse-cafe-banashankari/order ; 200
https://www.zomato.com/bangalore/smacznego-banashankari/order ; 200
https://www.zomato.com/bangalore/caf%C3%A9-down-the-alley-banashankari/order ; 200
https://www.zomato.com/bangalore/cafe-shuffle-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/the-coffee-shack-banashankari/order ; 200
https://www.zomato.com/bang

In [138]:
filtered_df = df_finals[(df_finals['menu'] != '') & (df_finals['menu'] != 'NA')]
filtered_df['Rest_Name'] = filtered_df['Rest_Name'].explode()
filtered_df

C:\Users\SNIGGDHA\AppData\Local\Temp/ipykernel_9988/2505624258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rest_Name'] = filtered_df['Rest_Name'].explode()


,Rest_id,URL,Rest_Name,Course_Type,menu,Menu Type,BestSeller,DishVotes,DishRating
15,1080,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Recommended,Afghani Soya Chaap,veg,BestSeller,15 votes,4.5
16,1080,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Recommended,Kadhai Paneer,veg,BestSeller,11 votes,4.5
17,1080,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Recommended,Paneer Lababdar,veg,BestSeller,13 votes,4.5
18,1080,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Recommended,Patiala Soya Chaap Masala,veg,BestSeller,10 votes,4.5
19,1080,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Recommended,Banjara Paneer Tikka,veg,BestSeller,42 votes,4.5
...,...,...,...,...,...,...,...,...,...
21308,2100,https://www.zomato.com/bangalore/polar-bear-1-...,Polar Bear,Ice Cream Scoops,Real Chikoo Ice Cream,veg,Not BestSeller,0,0.0
21309,400,https://www.zomato.com/bangalore/polar-bear-1-...,Polar Bear,Extras,Add On Brownie Cake,veg,Not BestSeller,0,0.0
21310,400,https://www.zomato.com/bangalore/polar-bear-1-...,Polar Bear,Extras,Add On Dark Chocolate Cake,veg,Not BestSeller,0,0.0
21311,400,https://www.zomato.com/bangalore/polar-bear-1-...,Polar Bear,Extras,Add On Red Velvet Cake,veg,Not BestSeller,0,0.0


In [149]:
filtered_df.groupby('Rest_Name')['Course_Type'].unique()
c= filtered_df[['Course_Type','Rest_Name']].drop_duplicates()

c.to_csv('C:/Users/SNIGGDHA/FP/test_list_course.csv',encoding ='UTF-8')

In [140]:
filtered_df.to_csv('C:/Users/SNIGGDHA/FP/test_check.csv',encoding ='UTF-8')

In [160]:
filtered_df['DishVotes'] = filtered_df['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)
#course_type_data['DishVotes'] = course_type_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)
        

C:\Users\SNIGGDHA\AppData\Local\Temp/ipykernel_9988/1832011759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DishVotes'] = filtered_df['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)


In [240]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
a =[]
selected_dishes =['Jeera Rice' ]

# Split the dataset by Course_Type category
course_type_groups = filtered_df[filtered_df['Rest_Name']=='Timepass Dinner'].groupby('Course_Type')
#course_type_groups.head(10)
for course_type, group in course_type_groups:
    # Create a subset DataFrame with relevant features
    subset = group[['menu', 'DishVotes', 'DishRating']]
    subset_reset_index = subset.reset_index(drop=True)
    similarity_matrix = cosine_similarity(subset_reset_index[['DishVotes', 'DishRating']])
    
    nn_model = NearestNeighbors(n_neighbors=10, metric='cosine')
    nn_model.fit(subset_reset_index[['DishVotes', 'DishRating']])

    for selected_dish in selected_dishes:
        # Look up the selected dish in the menu column
        selected_dish_row = subset_reset_index[subset_reset_index['menu'] == selected_dish]
      
        if selected_dish_row.empty:
            #print(f"Selected dish '{selected_dish}' not found in the '{course_type}' category.")
            continue
            
        selected_dish_row = selected_dish_row.drop_duplicates(keep=False)

        if not selected_dish_row.empty:
                selected_dish_idx = selected_dish_row.index[0]
                
                
        distances, nearest_indices = nn_model.kneighbors([subset_reset_index[['DishVotes', 'DishRating']].iloc[selected_dish_idx]])
        nearest_dish_idx = nearest_indices[0][1]
        
        nearest_scored_dish = subset_reset_index.iloc[nearest_dish_idx]['menu']
        nearest_scored_dish_list = [nearest_scored_dish]

        unique_dishes = list(set(nearest_scored_dish_list))

        for dish in unique_dishes:
            if dish not in a:
                a.append(dish)
    
# Print the selected dish and its unique nearest scored dishes
       # print(f"Nearest Scored Dishes: {', '.join(unique_dishes)}")
        
    


C:\ProgramData\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\ProgramData\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [272]:
print(a)
b = filtered_df[(filtered_df['Rest_Name'] == '1947') & (filtered_df['Course_Type'] == 'Recommended')]
contains_curd = b['menu'].str.contains('curd', case=False)
filtered_b = b[contains_curd]

# Print the filtered DataFrame
print(filtered_b['menu'].unique())

['Kaju Mattar Pulao', 'Palak Paneer']
['Curd Rice']


## Final Dish Recommendation

In [282]:
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommendation_model():
    # Step 1: Select Restaurant
    rest_name = input("Please enter the name of the restaurant: ")

    # Filter the dataset based on the selected restaurant name
    rest_data = filtered_df[filtered_df['Rest_Name'] == rest_name]
    #print(rest_data)
    if rest_data.empty:
        return "Restaurant not found. Please try again."
    

    rest_id = rest_data['Rest_id'].iloc[0]

    # Step 2: Select Menu Type
    unique_menu_types = rest_data['Menu Type'].unique()
    print("Available Menu Types:")
    for i, menu_type in enumerate(unique_menu_types, start=1):
        print(f"{i}. {menu_type}")

    #menu_type_choice = input("Please select a menu type: ")
    #menu_type = unique_menu_types[int(menu_type_choice) - 1]
    while True:
        menu_type_choice = input("Please select a menu type (veg/nonveg): ")
        menu_type = menu_type_choice.lower().strip()
        menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]

        if menu_type_data.empty:
            print("Invalid menu type. Please try again.")
            
        else:
            break
            
            
   # menu_type_choice = input("Please select a menu type (veg/nonveg): ")
    #menu_type = menu_type_choice.lower().strip()

    # Filter the dataset based on the selected restaurant and menu type
    #menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]
    #if menu_type_data.empty:
        #return "No menu found for the selected type. Please try again."

    # Step 3: Select BestSeller or Not
    #bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")
    #if bestseller_choice.lower() == 'yes':
        #filtered_data = menu_type_data[menu_type_data['BestSeller'] == 'BestSeller']
    #else:
        #filtered_data = menu_type_data
        
    while True:
        bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")
        if bestseller_choice.lower() == 'yes':
            filtered_data = menu_type_data[menu_type_data['BestSeller'] == 'BestSeller']
            if filtered_data.empty:
                print("No BestSeller Options available. Please select again.")
            else:
                break
        elif bestseller_choice.lower() == 'no':
            filtered_data = menu_type_data
            break
        else:
            print("Invalid choice. Please try again.")

    # Display the unique menu options that satisfy the selected conditions
    available_menu = filtered_data['menu'].unique()
    print("Available Menu Options:")
    for i, menu in enumerate(available_menu, start=1):
        print(f"{i}. {menu}")

    # Step 4: Select Menu Options
    menu_choices = input("Select the menu options (comma-separated menu names): ")
    menu_choices = [choice.strip() for choice in menu_choices.split(',')]

    selected_menus = [menu for menu in menu_choices if menu.strip() in available_menu]

    if not selected_menus:
        return "Menu selected is not in the list."

    # Step 5: Ask for Recommendation
    recommend_choice = input("Would you like to receive recommendations? (yes/no): ")
    if recommend_choice.lower() == 'yes':
        recommended_dishes=[]
        course_type_groups = rest_data.groupby('Course_Type')
        for course_type, group in course_type_groups:
                    subset = group[['menu', 'DishVotes', 'DishRating']]
                    subset_reset_index = subset.reset_index(drop=True)
                    similarity_matrix = cosine_similarity(subset_reset_index[['DishVotes', 'DishRating']])
                    if (subset_reset_index.shape[0] < 10):
                        n_neighbors = subset_reset_index.shape[0]
                        nn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
                        nn_model.fit(subset_reset_index[['DishVotes', 'DishRating']])
                    else:
                        nn_model = NearestNeighbors(n_neighbors=10, metric='cosine')
                        nn_model.fit(subset_reset_index[['DishVotes', 'DishRating']])
                        
                    for selected_dish in selected_menus:
        # Look up the selected dish in the menu column
                        selected_dish_row = subset_reset_index[subset_reset_index['menu'] == selected_dish]
                        
                        if selected_dish_row.empty:
            #print(f"Selected dish '{selected_dish}' not found in the '{course_type}' category.")
                            continue
            
                        selected_dish_row = selected_dish_row.drop_duplicates(keep=False)

                        if not selected_dish_row.empty:
                              selected_dish_idx = selected_dish_row.index[0]
                
                        #print(selected_dish_idx)
                        distances, nearest_indices = nn_model.kneighbors([subset_reset_index[['DishVotes', 'DishRating']].iloc[selected_dish_idx]])
                        n_actual_neighbors = min(n_neighbors, subset_reset_index.shape[0])
                        nearest_indices = nearest_indices[0][:n_actual_neighbors]
                        nearest_dish_idx = nearest_indices[1:]
        
                        nearest_scored_dish = subset_reset_index.iloc[nearest_dish_idx]['menu']
                        nearest_scored_dish_list = [nearest_scored_dish]

                        unique_dishes = nearest_scored_dish.values.tolist()
                        
                        for dish in unique_dishes:
                            #print(unique_dishes)
                            if dish not in recommended_dishes:
                                recommended_dishes.append(dish)

        return recommended_dishes

    return "Recommendation not requested. Thank you!"

recommendations = recommendation_model()
print(recommendations)

Please enter the name of the restaurant: 1947
Available Menu Types:
1. veg
Please select a menu type (veg/nonveg): veg
Do you want bestseller dishes only? (yes/no): yes
Available Menu Options:
1. Paneer Butter Masala
2. Sholay Paneer Tikka
3. Paneer Tikka Lababdar
4. Kadai Veg
5. Sabji Dum Biryani
6. Tawa Pulao
7. Dal Makhani
8. Kaju Matar Pulao
9. Jeera Rice
10. Curd Rice
11. Veg Kofta
12. Hakka Noodles
13. Hara Bhara Kabab
14. Butter Kulcha
15. Fried Rice
Select the menu options (comma-separated menu names): Paneer Butter Masala
Would you like to receive recommendations? (yes/no): yes
['Paneer Tikka Lababdar', 'Kadai Veg', 'Kadai Paneer', 'Butter Naan']


C:\ProgramData\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\ProgramData\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


# #####################################################   

In [128]:
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommendation_model():
    # Step 1: Select Restaurant
    rest_name = input("Please enter the name of the restaurant: ")
    print(rest_name)
    # Filter the dataset based on the selected restaurant name
    rest_data = filtered_df[filtered_df['Rest_Name'] == rest_name]
    #print(rest_data)
    if rest_data.empty:
        #print("2")
        return "Restaurant not found. Please try again."
    
    #print("1")
    rest_id = rest_data['Rest_id'].iloc[0]
    #print(rest_id)

    # Step 2: Select Menu Type
    unique_menu_types = rest_data['Menu Type'].unique()
    print("Available Menu Types:")
    for i, menu_type in enumerate(unique_menu_types, start=1):
        print(f"{i}. {menu_type}")

    #menu_type_choice = input("Please select a menu type: ")
    #menu_type = unique_menu_types[int(menu_type_choice) - 1]
    while True:
        menu_type_choice = input("Please select a menu type (veg/nonveg): ")
        menu_type = menu_type_choice.lower().strip()
        menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]

        if menu_type_data.empty:
            print("Invalid menu type. Please try again.")
            
        else:
            break
            
    course_types = menu_type_data['Course_Type'].unique()
    print("Available Course Types:")
    for i, course_type in enumerate(course_types, start=1):
            print(f"{i}. {course_type}")
            
    while True:
            course_type_choice = input("Please select a course type: ")
            course_type_data = menu_type_data[menu_type_data['Course_Type'] == course_type_choice]
            

            if course_type_data.empty:
                print("Invalid course type. Please try again.")
            else: 
                break
            
    

    bestseller_values = course_type_data['BestSeller'].unique()

    if len(bestseller_values) > 1:
        while True:
            bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")

            if bestseller_choice.lower() == 'yes':
                filtered_data = course_type_data[course_type_data['BestSeller'] == 'BestSeller']

                if filtered_data.empty:
                    print("No BestSeller Options available. Please select again.")
                else:
                    break

            elif bestseller_choice.lower() == 'no':
                filtered_data = menu_type_data
                break

            else:
                print("Invalid choice. Please try again.")
    else:
    # Use the unique value for further filtering
         bestseller_value = bestseller_values[0]
         filtered_data = course_type_data[course_type_data['BestSeller'] == bestseller_value]
    
    '''
    grouped_data = df_finals.groupby('Course_Type')['menu'].unique()
    
    print("Available Menu Options:")
    for course_type, menus in grouped_data.items():
        print("\n")
        print('course_type :' ,course_type)
        for i, menu in enumerate(menus, start=1):
            print(f"   {i}. {menu}")
    '''        
    available_menu = filtered_data['menu'].unique()
    print("Available Menu Options:")
    for i, menu in enumerate(available_menu, start=1):
        print(f"{i}. {menu}")

    while True:
        menu_choices = input("Select the menu option : ")
        menu_choices = [choice.strip() for choice in menu_choices.split(',')]
        selected_menus = [menu for menu in menu_choices if menu in available_menu]

        if course_type_data.empty:
                print("Menu selected is not in the list. Please try again.")
        else:
                break

    # Step 5: Ask for Recommendation
    recommend_choice = input("Would you like to receive recommendations? (yes/no): ")
    if recommend_choice.lower() == 'yes':
        # Filter the dataset based on selected menu options for selected data
        selected_data = filtered_data[filtered_data['menu'].isin(selected_menus)]
        selected_data = selected_data.drop_duplicates()
        if len(selected_data['Rest_id'].unique()) > 1:
                selected_data = selected_data.head(1)

        #selected_data['DishVotes'] = selected_data['DishVotes'].str.extract('(\d+)').astype(float)
        #menu_type_data['DishVotes'] = menu_type_data['DishVotes'].str.extract('(\d+)').astype(float)
        
        selected_data['DishVotes'] = selected_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)
        course_type_data['DishVotes'] = course_type_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)
        course_type_data =  course_type_data.drop_duplicates()
        selected_features = selected_data[['DishVotes', 'DishRating']].fillna(0)
        selected_similarity_matrix = cosine_similarity(selected_features)
        # Calculate the similarity score based on DishVotes and DishRating for menu_type_data
        menu_type_features = course_type_data[['DishVotes', 'DishRating']].fillna(0)
        menu_type_similarity_matrix = cosine_similarity(menu_type_features)

        # Retrieve the recommended dishes with scores
        recommended_dishes = set()
        for i in range(selected_similarity_matrix.shape[0]):
            if selected_data.iloc[i]['menu'] in selected_menus:
                similar_indices = np.argsort(selected_similarity_matrix[i])[::-1]  # Sort in descending order of similarity
                similar_dishes = list(set(course_type_data.iloc[similar_indices]['menu'].tolist()))
                for dish in similar_dishes:
                    if dish not in selected_menus:
                        recommended_dishes.add((dish))
                        if len(recommended_dishes) == 20:
                            break
                if len(recommended_dishes) == 20:
                    break

        recommended_dishes = list(recommended_dishes)
        recommended_dishes.sort(key=lambda x: x[1], reverse=True)

        return recommended_dishes

    return "Recommendation not requested. Thank you!"
recommendations = recommendation_model()
print(recommendations)

Please enter the name of the restaurant: Timepass Dinner
Timepass Dinner
Available Menu Types:
1. veg
Please select a menu type (veg/nonveg): veg
Available Course Types:
1. Recommended
2. Shorba
3. Chinese Soups
4. Accompaniments
5. Rangde Punjabi
6. Chini Suruata
7. Punjabi Kada Kebab
8. Soya Chaap
9. Patiala Express
10. Tandoori Ki Aanch Se
11. Parathon Ka Swad
12. Sada Bahar Sabziyan
13. Dal Ka Pakwan
14. Chawal Ki Farmaish
15. Oye Lassi
16. Meetha Pakwan
17. Ice Cream & Milkshake
18. Chinese Rice & Noodles
Please select a course type: Dal Ka Pakwan
Do you want bestseller dishes only? (yes/no): yes
Available Menu Options:
1. Dal Makhni
2. Punjabi Dal Tadka
Select the menu option : Dal Makhni
Would you like to receive recommendations? (yes/no): yes
['Dal Fry']


C:\Users\SNIGGDHA\AppData\Local\Temp/ipykernel_9988/2429949843.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_type_data['DishVotes'] = course_type_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)


In [93]:
A = filtered_df[(filtered_df['Rest_Name'] == 'Polar Bear') & (filtered_df['Menu Type'] == 'veg')  & (filtered_df['BestSeller'] == 'BestSeller')
                                                                                                &(filtered_df['Course_Type'] == 'Recommended')]

In [95]:
A.to_csv('C:/Users/SNIGGDHA/FP/test_check.csv',encoding ='UTF-8')

In [47]:

for section in container:
    Course_type=[]
    Menu = []
    Menu_Type = []
    BestSeller = []
    Votes = []
    Dish_Rating = []
    h4_tags_course = section.find('h4', class_=lambda x: x and 'sc-1hp8d8a-0' in x.split())
    if h4_tags_course is None:
        continue
    else:
        h4_tags_text = h4_tags_course.text.strip()
        if h4_tags_text:
            h4_tags_text_rep = h4_tags_text.replace("[", "").replace("]", "")
            Course_type.append(h4_tags_text_rep)
        

    container_menu = section.find_all("div", class_=False, attrs=False)

    if len(container_menu) == 0:
        continue
    else:
        for div in container_menu:
            h4_tags = div.find("h4", class_="sc-1s0saks-15 iSmBPS")
            if h4_tags:
                for h4 in h4_tags:
                    Menu.append(h4)
            else:
                Menu.append("NA")

            type_tag = div.find('div', attrs={'type': True})
            if type_tag:
                type_value = type_tag['type']
                Menu_Type.append(type_value)
            else:
                Menu_Type.append("NA")

            if div and 'BESTSELLER' in div.text:
                BestSeller.append('BestSeller')
            else:
                BestSeller.append('Not BestSeller')

            def find_votes_div(tag):
                return tag.name == 'span' and 'votes' in tag.text

            votes_div = div.find(find_votes_div)
            if votes_div:
                votes_text = votes_div.text.strip()
                Votes.append(votes_text)
            else:
                votes_text = 0
                Votes.append(votes_text)

            div_tag = div.find('div', class_='sc-z30xqq-3 bewuUV')
            count = 0
            if div_tag:
                i_tags = div_tag.find_all('i', color='#F3C117')
                
                for i in i_tags:
                    if i.find('title') and i.find('title').text == 'star-fill':
                        count += 1
                    else:
                        count += 0.5
                Dish_Rating.append(count)
            else:
                count += 0
                Dish_Rating.append(count)

    df_iteration = pd.DataFrame({
        'Rest_id' : Rest_id * len(Menu),
        'URL': base_url * len(Menu),
        'Rest_Name': [Rest_Name] * len(Menu),
        'Course_Type' : Course_type * len(Menu),
        'menu': Menu,
        'Menu Type': Menu_Type,
        'BestSeller': BestSeller,
        'DishVotes': Votes,
        'DishRating': Dish_Rating
    })
    
    df_finals = pd.concat([df_finals, df_iteration], ignore_index=True) 

print(df_finals)


      Rest_id                                                URL  \
0         840  https://www.zomato.com/bangalore/rosewood-inte...   
1         840  https://www.zomato.com/bangalore/rosewood-inte...   
2         840  https://www.zomato.com/bangalore/rosewood-inte...   
3         840  https://www.zomato.com/bangalore/rosewood-inte...   
4         840  https://www.zomato.com/bangalore/rosewood-inte...   
...       ...                                                ...   
2395      105  https://www.zomato.com/bangalore/rosewood-inte...   
2396      105  https://www.zomato.com/bangalore/rosewood-inte...   
2397      105  https://www.zomato.com/bangalore/rosewood-inte...   
2398      105  https://www.zomato.com/bangalore/rosewood-inte...   
2399      105  https://www.zomato.com/bangalore/rosewood-inte...   

                                              Rest_Name       Course_Type  \
0     6    Rosewood International Hotel - Bar & Rest...  [Special Combos]   
1     6    Rosewood Internati

In [53]:
filtered_df = df_finals

In [60]:
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommendation_model():
    # Step 1: Select Restaurant
    rest_name = input("Please enter the name of the restaurant: ")
    print(rest_name)
    # Filter the dataset based on the selected restaurant name
    rest_data = filtered_df[filtered_df['Rest_Name'] == rest_name]
    print(rest_data)
    if rest_data.empty:
        print("2")
        return "Restaurant not found. Please try again."
    
    print("1")
    rest_id = rest_data['Rest_id'].iloc[0]
    print(rest_id)

    # Step 2: Select Menu Type
    unique_menu_types = rest_data['Menu Type'].unique()
    print("Available Menu Types:")
    for i, menu_type in enumerate(unique_menu_types, start=1):
        print(f"{i}. {menu_type}")

    #menu_type_choice = input("Please select a menu type: ")
    #menu_type = unique_menu_types[int(menu_type_choice) - 1]
    while True:
        menu_type_choice = input("Please select a menu type (veg/nonveg): ")
        menu_type = menu_type_choice.lower().strip()
        menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]

        if menu_type_data.empty:
            print("Invalid menu type. Please try again.")
            
        else:
            break
            
            
   # menu_type_choice = input("Please select a menu type (veg/nonveg): ")
    #menu_type = menu_type_choice.lower().strip()

    # Filter the dataset based on the selected restaurant and menu type
    #menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]
    #if menu_type_data.empty:
        #return "No menu found for the selected type. Please try again."

    # Step 3: Select BestSeller or Not
    #bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")
    #if bestseller_choice.lower() == 'yes':
        #filtered_data = menu_type_data[menu_type_data['BestSeller'] == 'BestSeller']
    #else:
        #filtered_data = menu_type_data
        
    while True:
        bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")
        if bestseller_choice.lower() == 'yes':
            filtered_data = menu_type_data[menu_type_data['BestSeller'] == 'BestSeller']
            if filtered_data.empty:
                print("No BestSeller Options available. Please select again.")
            else:
                break
        elif bestseller_choice.lower() == 'no':
            filtered_data = menu_type_data
            break
        else:
            print("Invalid choice. Please try again.")

    # Display the unique menu options that satisfy the selected conditions
    available_menu = filtered_data['menu'].unique()
    print("Available Menu Options:")
    for i, menu in enumerate(available_menu, start=1):
        print(f"{i}. {menu}")

    # Step 4: Select Menu Options
    menu_choices = input("Select the menu option : ")
    menu_choices = [choice.strip() for choice in menu_choices.split(',')]

    selected_menus = [menu for menu in menu_choices if menu in available_menu]

    if not selected_menus:
        return "Menu selected is not in the list."

    # Step 5: Ask for Recommendation
    recommend_choice = input("Would you like to receive recommendations? (yes/no): ")
    if recommend_choice.lower() == 'yes':
        # Filter the dataset based on selected menu options for selected data
        selected_data = filtered_data[filtered_data['menu'].isin(selected_menus)]
        Course_Type = selected_data['Course_Type'].unique()
        print (Course_Type)
        #selected_data['DishVotes'] = selected_data['DishVotes'].str.extract('(\d+)').astype(float)
        #menu_type_data['DishVotes'] = menu_type_data['DishVotes'].str.extract('(\d+)').astype(float)
        
        selected_data['DishVotes'] = selected_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)
        menu_type_data['DishVotes'] = menu_type_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)
        Course_type_data = menu_type_data[menu_type_data['Course_Type'] == Course_Type]

        # Calculate the similarity score based on DishVotes and DishRating for selected data
        selected_features = selected_data[['DishVotes', 'DishRating']].fillna(0)
        selected_similarity_matrix = cosine_similarity(selected_features)

        # Calculate the similarity score based on DishVotes and DishRating for menu_type_data
        menu_type_features = Course_type_data[['DishVotes', 'DishRating']].fillna(0)
        menu_type_similarity_matrix = cosine_similarity(menu_type_features)

        # Retrieve the recommended dishes with scores
        recommended_dishes = set()
        for i in range(selected_similarity_matrix.shape[0]):
            if selected_data.iloc[i]['menu'] in selected_menus:
                similar_indices = np.argsort(selected_similarity_matrix[i])[::-1]  # Sort in descending order of similarity
                similar_dishes = list(set(menu_type_data.iloc[similar_indices]['menu'].tolist()))
                for dish in similar_dishes:
                    if dish not in selected_menus:
                        recommended_dishes.add((dish))
                        if len(recommended_dishes) == 20:
                            break
                if len(recommended_dishes) == 20:
                    break

        recommended_dishes = list(recommended_dishes)
        recommended_dishes.sort(key=lambda x: x[1], reverse=True)

        return recommended_dishes

    return "Recommendation not requested. Thank you!"
recommendations = recommendation_model()
print(recommendations)

Please enter the name of the restaurant: Rosewood International Hotel - Bar & Restaurant
Rosewood International Hotel - Bar & Restaurant


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# OLD Version

In [175]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

Header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
}

url = [
    #'https://www.zomato.com/bangalore/rosewood-international-hotel-bar-restaurant-banashankari-bangalore?',
    'https://www.zomato.com/bangalore/rosewood-international-hotel-bar-restaurant-mysore-road-bangalore?context=eyJzZSI6eyJlIjpbIjU5MDkwIiwiMTgzODI5NDQiLCIxODIyNDY3NiIsIjU5Mjg5IiwxODM3MzM4Nl0sInQiOiJSZXN0YXVyYW50cyBpbiBCYW5hc2hhbmthcmkgc2VydmluZyBCdWZmZXQifX0='
]
for i in url:
    original_url = i
    Rest_Name = df[df['url'] == original_url]['name']
    Rest_id = df[df['url'] == original_url]['Rest_id'].iloc[0]
    #top_restid = df['restid'].iloc[0]
    #print("Top RestID:", top_restid)
    end_index = i.find("?")

    if end_index != -1:
        base_url = i[:end_index] + "/order"
    else:
        base_url = i

    response = requests.get(base_url, headers=Header)
    print(base_url, ';', response.status_code)

    soup_per_page = bs(response.content, 'html.parser')
    
    h4_tags = soup_per_page.find_all('h4', class_=lambda x: x and 'sc-1hp8d8a-0' in x.split())
    print(h4_tags)

    section_class_names = set()
    for h4_tag in h4_tags:
        section_tag = h4_tag.find_parent('section')
        if section_tag:
            class_names = section_tag.get('class', [])
            section_class_names.update(class_names)

    print(section_class_names)


https://www.zomato.com/bangalore/rosewood-international-hotel-bar-restaurant-mysore-road-bangalore/order ; 200
{'giFCQq', 'sc-eBipZS'}


In [220]:
h4_tags = soup_per_page.find_all('h4', class_=lambda x: x and 'sc-1hp8d8a-0' in x.split())
#print(h4_tags)

class_attributes = [tag['class'] for tag in h4_tags]
unique_class_attributes = set([attr for sublist in class_attributes for attr in sublist])

#for class_attr in unique_class_attributes:
#print(unique_class_attributes)


h4_tags = soup_per_page.find_all('h4', class_=lambda x: x and 'sc-1hp8d8a-0' in x.split())
#print(h4_tags)

class_attributes = [' '.join(tag['class']) for tag in h4_tags]
unique_class_attributes = set (class_attributes)
#print(unique_class_attributes)

from bs4 import BeautifulSoup, Tag

h4_tags = soup_per_page.find_all('h4', class_=unique_class_attributes)
parent_tags = [tag.parent for tag in h4_tags]

# Convert parent_tags to BeautifulSoup object
parent_soup = BeautifulSoup("", "html.parser")
for tag in parent_tags:
    parent_soup.append(tag)
# Now parent_tags is of h4_tag datatype

print(parent_soup)


Menu=[]
food_headers=[]
for parent_tag in parent_soup:
            food_header = parent_soup.find("h4", class_="sc-1hp8d8a-0 sc-bjPkoM iscyhv")
    
            if food_header:
                for h4 in food_header:
                    food_headers.append(h4)
            else:
                food_headers.append("NA")
                
            h4_tags = parent_soup.find("h4", class_="sc-1s0saks-15 iSmBPS")
            if h4_tags:
                for h4 in h4_tags:
                    Menu.append(h4)
            else:
                Menu.append("NA")
print(Menu,food_headers)






[] []


##################

In [1]:
import json
import pandas as pd
import string
import spacy

In [2]:
df = pd.read_csv(r"C:\Users\SNIGGDHA\Downloads\zomato.csv\zomato.csv")

In [3]:
df['Rest_id'] = range (1,len(df)+1)

In [4]:
filtered_df = df[(df['Rest_id'] >= 1) & (df['Rest_id'] <= 1000)]

url = filtered_df['url'].tolist()

In [5]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

df_finals = pd.DataFrame()  # Create an empty DataFrame outside the loop

Header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
}

# df = pd.read_csv(r"C:\Users\SNIGGDHA\Downloads\zomato.csv\zomato.csv")


for i in url:
    original_url = i
    Rest_Name = df[df['url'] == original_url]['name']
    Rest_id = df[df['url'] == original_url]['Rest_id'].iloc[0]
    end_index = i.find("?")

    if end_index != -1:
        base_url = i[:end_index] + "/order"
    else:
        base_url = i

    response = requests.get(base_url, headers=Header)
    print(base_url, ';', response.status_code)

    soup_per_page = bs(response.content, 'html.parser')

    container = soup_per_page.find_all("div", class_=False, attrs=False)

    Menu = []
    Menu_Type = []
    BestSeller = []
    Votes = []
    Dish_Rating = []
    if len(container) == 0:
        continue
    else:
        for div in container:
            h4_tags = div.find("h4", class_="sc-1s0saks-15 iSmBPS")
            if h4_tags:
                for h4 in h4_tags:
                    Menu.append(h4)
            else:
                Menu.append("NA")

            type_tag = div.find('div', attrs={'type': True})
            if type_tag:
                type_value = type_tag['type']
                Menu_Type.append(type_value)
            else:
                Menu_Type.append("NA")

            if div and 'BESTSELLER' in div.text:
                BestSeller.append('BestSeller')
            else:
                BestSeller.append('Not BestSeller')

            def find_votes_div(tag):
                return tag.name == 'span' and 'votes' in tag.text

            votes_div = div.find(find_votes_div)
            if votes_div:
                votes_text = votes_div.text.strip()
                Votes.append(votes_text)
            else:
                votes_text = 0
                Votes.append(votes_text)

            div_tag = div.find('div', class_='sc-z30xqq-3 bewuUV')
            count = 0
            if div_tag:
                i_tags = div_tag.find_all('i', color='#F3C117')
                
                for i in i_tags:
                    if i.find('title') and i.find('title').text == 'star-fill':
                        count += 1
                    else:
                        count += 0.5
                Dish_Rating.append(count)
            else:
                count += 0
                Dish_Rating.append(count)

    df_iteration = pd.DataFrame({
        'Rest_id' : Rest_id * len(Menu),
        'URL': base_url * len(Menu),
        'Rest_Name': [Rest_Name] * len(Menu),
        'menu': Menu,
        'Menu Type': Menu_Type,
        'BestSeller': BestSeller,
        'DishVotes': Votes,
        'DishRating': Dish_Rating
    })
    
    df_finals = pd.concat([df_finals, df_iteration], ignore_index=True) 

print(df_finals)


https://www.zomato.com/bangalore/jalsa-banashankari/order ; 200
https://www.zomato.com/bangalore/spice-elephant-banashankari/order ; 200
https://www.zomato.com/SanchurroBangalore/order ; 200
https://www.zomato.com/bangalore/addhuri-udupi-bhojana-banashankari/order ; 200
https://www.zomato.com/bangalore/grand-village-basavanagudi/order ; 200
https://www.zomato.com/bangalore/timepass-dinner-basavanagudi/order ; 200
https://www.zomato.com/bangalore/rosewood-international-hotel-bar-restaurant-mysore-road-bangalore/order ; 200
https://www.zomato.com/bangalore/onesta-banashankari/order ; 200
https://www.zomato.com/bangalore/penthouse-cafe-banashankari/order ; 200
https://www.zomato.com/bangalore/smacznego-banashankari/order ; 200
https://www.zomato.com/bangalore/caf%C3%A9-down-the-alley-banashankari/order ; 200
https://www.zomato.com/bangalore/cafe-shuffle-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/the-coffee-shack-banashankari/order ; 200
https://www.zomato.com/bang

https://www.zomato.com/bangalore/naveen-kabab-biriyani-mane-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/katriguppe-donne-biryani-banashankari/order ; 200
https://www.zomato.com/bangalore/hari-super-sandwich-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/atithi-point-ande-ka-funda-banashankari/order ; 200
https://www.zomato.com/bangalore/just-bake-3-banashankari/order ; 200
https://www.zomato.com/bangalore/dharwad-line-bazaar-mishra-pedha-banashankari/order ; 200
https://www.zomato.com/bangalore/cake-bite-banashankari/order ; 200
https://www.zomato.com/bangalore/aarushs-food-plaza-banashankari/order ; 200
https://www.zomato.com/bangalore/wood-stove-banashankari/order ; 200
https://www.zomato.com/bangalore/kulfi-more-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/kannadigas-karavali-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/k27-the-pub-banashankari-bangalore/order ; 200
https://www.zomato.com/bang

https://www.zomato.com/bangalore/vietnamese-kitchen-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/smacznego-banashankari/order ; 200
https://www.zomato.com/bangalore/c-corner-kumaraswamy-layout/order ; 200
https://www.zomato.com/bangalore/paratha-merchant-kumaraswamy-layout/order ; 200
https://www.zomato.com/bangalore/north-rasoe-1-kumaraswamy-layout-bangalore/order ; 200
https://www.zomato.com/bangalore/inchara-restaurant-basavanagudi-bangalore/order ; 200
https://www.zomato.com/bangalore/toscano-jayanagar/order ; 200
https://www.zomato.com/bangalore/lord-of-the-kitchen-1-kumaraswamy-layout-bangalore/order ; 200
https://www.zomato.com/bangalore/stoned-monkey-banashankari/order ; 200
https://www.zomato.com/bangalore/central-jail-restaurant-jayanagar/order ; 200
https://www.zomato.com/bangalore/bella-jayanagar-bangalore/order ; 200
https://www.zomato.com/bangalore/vennela-andhra-meals-jayanagar-bangalore/order ; 200
https://www.zomato.com/bangalore/new-prashanth-ho

https://www.zomato.com/bangalore/dakshin-grand-banashankari/order ; 200
https://www.zomato.com/bangalore/sandwichwallas-1-jayanagar/order ; 200
https://www.zomato.com/bangalore/chai-mane-basavanagudi/order ; 200
https://www.zomato.com/bangalore/slv-upachar-kumaraswamy-layout/order ; 200
https://www.zomato.com/bangalore/waffle-a-go-go-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/cross-road-cafe-jayanagar/order ; 200
https://www.zomato.com/bangalore/anand-donne-biriyani-jayanagar/order ; 200
https://www.zomato.com/bangalore/seven-star-banashankari/order ; 200
https://www.zomato.com/bangalore/stop-n-joy-banashankari/order ; 200
https://www.zomato.com/bangalore/sri-yaksha-shiv-sagar-banashankari/order ; 200
https://www.zomato.com/bangalore/fudge-2-basavanagudi/order ; 200
https://www.zomato.com/bangalore/just-bunny-jayanagar/order ; 200
https://www.zomato.com/bangalore/bakers-town-kumaraswamy-layout/order ; 200
https://www.zomato.com/bangalore/shree-mahalakshmi-sweet

https://www.zomato.com/bangalore/dakshin-cafe-1-kumaraswamy-layout-bangalore/order ; 200
https://www.zomato.com/bangalore/mr-parathewala-jayanagar-bangalore/order ; 200
https://www.zomato.com/bangalore/kaulige-millet-corner-basavanagudi-bangalore/order ; 200
https://www.zomato.com/bangalore/ifruit-ice-cream-kumaraswamy-layout-bangalore/order ; 200
https://www.zomato.com/bangalore/havmor-basavanagudi-bangalore/order ; 200
https://www.zomato.com/bangalore/grand-food-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/dining-hut-kumaraswamy-layout-bangalore/order ; 200
https://www.zomato.com/bangalore/7-hills-garden-restaurant-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/desi-swaad-kumaraswamy-layout-bangalore/order ; 200
https://www.zomato.com/bangalore/yummy-momos-kumaraswamy-layout-bangalore/order ; 200
https://www.zomato.com/bangalore/masale-daan-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/sri-manjunatha-tandoori-corner-ut

https://www.zomato.com/bangalore/me-and-my-cake-banashankari/order ; 200
https://www.zomato.com/bangalore/mukesh-sweet-centre-banashankari/order ; 200
https://www.zomato.com/bangalore/darkolates-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/sai-super-sandwich-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/sweet-truth-banashankari/order ; 200
https://www.zomato.com/bangalore/waffle-a-go-go-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/gopi-ice-creams-banashankari-bangalore/order ; 200
https://www.zomato.com/bangalore/sri-venkateshwara-sweet-meat-stall-basavanagudi/order ; 200
https://www.zomato.com/bangalore/baskin-robbins-basavanagudi/order ; 200
https://www.zomato.com/bangalore/redberrys-basavanagudi/order ; 200
https://www.zomato.com/bangalore/amande-patisserie-basavanagudi/order ; 200
https://www.zomato.com/bangalore/srinivasa-brahmins-bakery-basavanagudi-bangalore/order ; 200
https://www.zomato.com/bangalore/jain-bake

ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [6]:
df_finals.to_csv('C:/Users/SNIGGDHA/FP/test_check.csv',encoding ='UTF-8')

In [8]:
filtered_df = df_finals[(df_finals['menu'] != '') & (df_finals['menu'] != 'NA')]

In [133]:
filtered_df['Rest_Name'] = filtered_df['Rest_Name'].explode()
filtered_df

C:\Users\SNIGGDHA\AppData\Local\Temp/ipykernel_2324/3478809311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rest_Name'] = filtered_df['Rest_Name'].explode()


,Rest_id,URL,Rest_Name,menu,Menu Type,BestSeller,DishVotes,DishRating
26,1056,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Paneer Lababdar,veg,BestSeller,13 votes,4.5
27,1056,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Paneer Lababdar,veg,BestSeller,13 votes,4.5
28,1056,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Afghani Soya Chaap,veg,BestSeller,15 votes,4.5
29,1056,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Punjabi Paneer Bhurji,veg,BestSeller,0,0.0
30,1056,https://www.zomato.com/bangalore/timepass-dinn...,Timepass Dinner,Banjara Paneer Tikka,veg,BestSeller,41 votes,4.5
...,...,...,...,...,...,...,...,...
30436,84508,https://www.zomato.com/bangalore/anna-kuteera-...,Anna Kuteera,Kesar Falooda,veg,Not BestSeller,0,0.0
30437,84508,https://www.zomato.com/bangalore/anna-kuteera-...,Anna Kuteera,Royal Falooda,veg,Not BestSeller,0,0.0
30438,84508,https://www.zomato.com/bangalore/anna-kuteera-...,Anna Kuteera,Carrot Halwa,veg,Not BestSeller,25 votes,4.5
30439,84508,https://www.zomato.com/bangalore/anna-kuteera-...,Anna Kuteera,Jamoon,veg,Not BestSeller,19 votes,4.5


## Dish Recommendation Model

In [152]:
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommendation_model():
    # Step 1: Select Restaurant
    rest_name = input("Please enter the name of the restaurant: ")

    # Filter the dataset based on the selected restaurant name
    rest_data = filtered_df[filtered_df['Rest_Name'] == rest_name]
    #print(rest_data)
    if rest_data.empty:
        return "Restaurant not found. Please try again."
    

    rest_id = rest_data['Rest_id'].iloc[0]

    # Step 2: Select Menu Type
    unique_menu_types = rest_data['Menu Type'].unique()
    print("Available Menu Types:")
    for i, menu_type in enumerate(unique_menu_types, start=1):
        print(f"{i}. {menu_type}")

    #menu_type_choice = input("Please select a menu type: ")
    #menu_type = unique_menu_types[int(menu_type_choice) - 1]
    while True:
        menu_type_choice = input("Please select a menu type (veg/nonveg): ")
        menu_type = menu_type_choice.lower().strip()
        menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]

        if menu_type_data.empty:
            print("Invalid menu type. Please try again.")
            
        else:
            break
            
            
   # menu_type_choice = input("Please select a menu type (veg/nonveg): ")
    #menu_type = menu_type_choice.lower().strip()

    # Filter the dataset based on the selected restaurant and menu type
    #menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]
    #if menu_type_data.empty:
        #return "No menu found for the selected type. Please try again."

    # Step 3: Select BestSeller or Not
    #bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")
    #if bestseller_choice.lower() == 'yes':
        #filtered_data = menu_type_data[menu_type_data['BestSeller'] == 'BestSeller']
    #else:
        #filtered_data = menu_type_data
        
    while True:
        bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")
        if bestseller_choice.lower() == 'yes':
            filtered_data = menu_type_data[menu_type_data['BestSeller'] == 'BestSeller']
            if filtered_data.empty:
                print("No BestSeller Options available. Please select again.")
            else:
                break
        elif bestseller_choice.lower() == 'no':
            filtered_data = menu_type_data
            break
        else:
            print("Invalid choice. Please try again.")

    # Display the unique menu options that satisfy the selected conditions
    available_menu = filtered_data['menu'].unique()
    print("Available Menu Options:")
    for i, menu in enumerate(available_menu, start=1):
        print(f"{i}. {menu}")

    # Step 4: Select Menu Options
    menu_choices = input("Select the menu options (comma-separated menu names): ")
    menu_choices = [choice.strip() for choice in menu_choices.split(',')]

    selected_menus = [menu for menu in menu_choices if menu in available_menu]

    if not selected_menus:
        return "Menu selected is not in the list."

    # Step 5: Ask for Recommendation
    recommend_choice = input("Would you like to receive recommendations? (yes/no): ")
    if recommend_choice.lower() == 'yes':
        # Filter the dataset based on selected menu options for selected data
        selected_data = filtered_data[filtered_data['menu'].isin(selected_menus)]
        #print (selected_data)
        #selected_data['DishVotes'] = selected_data['DishVotes'].str.extract('(\d+)').astype(float)
        #menu_type_data['DishVotes'] = menu_type_data['DishVotes'].str.extract('(\d+)').astype(float)
        
        selected_data['DishVotes'] = selected_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)
        menu_type_data['DishVotes'] = menu_type_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)


        # Calculate the similarity score based on DishVotes and DishRating for selected data
        selected_features = selected_data[['DishVotes', 'DishRating']].fillna(0)
        selected_similarity_matrix = cosine_similarity(selected_features)

        # Calculate the similarity score based on DishVotes and DishRating for menu_type_data
        menu_type_features = menu_type_data[['DishVotes', 'DishRating']].fillna(0)
        menu_type_similarity_matrix = cosine_similarity(menu_type_features)

        # Retrieve the recommended dishes with scores
        recommended_dishes = set()
        for i in range(selected_similarity_matrix.shape[0]):
            if selected_data.iloc[i]['menu'] in selected_menus:
                similar_indices = np.argsort(selected_similarity_matrix[i])[::-1]  # Sort in descending order of similarity
                similar_dishes = list(set(menu_type_data.iloc[similar_indices]['menu'].tolist()))
                for dish in similar_dishes:
                    if dish not in selected_menus:
                        recommended_dishes.add((dish))
                        if len(recommended_dishes) == 20:
                            break
                if len(recommended_dishes) == 20:
                    break

        recommended_dishes = list(recommended_dishes)
        recommended_dishes.sort(key=lambda x: x[1], reverse=True)

        return recommended_dishes

    return "Recommendation not requested. Thank you!"
recommendations = recommendation_model()
print(recommendations)

Please enter the name of the restaurant: Empire Restaurant
Available Menu Types:
1. non-veg
2. veg
Please select a menu type (veg/nonveg): veg
Do you want bestseller dishes only? (yes/no): yes
Available Menu Options:
1. Ghee Rice
Select the menu options (comma-separated menu names): Ghee Rice
Would you like to receive recommendations? (yes/no): yes
['Tender Coconut Pudding']


C:\Users\SNIGGDHA\AppData\Local\Temp/ipykernel_2324/2207119322.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  menu_type_data['DishVotes'] = menu_type_data['DishVotes'].apply(lambda x: re.findall(r'\d+', x)[0] if isinstance(x, str) and 'votes' in x else x)


In [143]:
filtered_df[filtered_df['Menu Type']=='non-veg']['Rest_Name'].unique()
#filtered_df['Menu Type'].unique()

array(['Rosewood International Hotel - Bar & Restaurant',
       'CafÃ\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley',
       'Cafe Shuffle', 'Caf-Eleven', 'Hide Out Cafe', 'Sweet Truth',
       'Ovenstory Pizza', 'Faasos', 'Behrouz Biryani', 'Szechuan Dragon',
       'Empire Restaurant', "McDonald's", "Domino's Pizza",
       'Kitchen Garden', 'Recipe', 'Beijing Bites', 'FreshMenu',
       'Banashankari Donne Biriyani', 'Five Star Chicken', 'Kabab Magic',
       'Burger King', 'The Good Bowl', 'Rolls On Wheels',
       'Devanna Dum Biriyani Centre', 'Vaishali Deluxe', 'Meghana Foods',
       'Chickpet Donne Biryani House', 'Nandhini Deluxe',
       'Naveen Kabab & Biriyani Mane', "Aarush's Food Plaza",
       'Kannadigas Karavali', 'Roll Magic Fast Food', 'House Of Kebabs',
       'Roll N Rock', 'BOX8- Desi Meals', 'Imperial Restaurant', 'Canton',
       'Aramane Donne Biryani', 'Tandoori Knight', 'Paradise Premium',
       'Grazers', 'Just

## Trial

In [19]:
def filter_menu(rest_name, rest_id, menu_type, bestseller):
    # Filter the dataset based on the provided criteria
    filtered_data = filtered_df[(filtered_df['Rest_Name'] == rest_name) &
                       (filtered_df['Rest_id'] == rest_id) &
                       (filtered_df['Menu Type'] == menu_type) &
                       (filtered_df['BestSeller'] == bestseller)]
    
    # Extract the menu list from the filtered data
    menu_list = filtered_data['menu'].tolist()
    
    return menu_list


In [20]:
filter_menu('Anna Kuteera' ,84508,'veg','Not BestSeller')

['Rava Idly (1 Pc)',
 'Mushroom Fried Rice',
 'Crispy Babycorn',
 'Idly (2 Pc)',
 'Dahi Puri',
 'Paneer Manchurian',
 'Onion Dosa',
 'Open Butter Masala',
 'Corn Kabab',
 'Harabhara Kabab',
 'Sev Puri',
 'Mango Fruit Milkshake',
 'Pani Puri',
 'Samosa Masala',
 'Jamoon',
 'Tomato Soup',
 'Coffee',
 'Veg Manchow Soup',
 'Carrot Halwa',
 'Buns(1pcs)',
 'Masala Vada(2)',
 '1 Idly 1 Vada',
 'Idly (2 Pc)',
 'Rava Idly (1 Pc)',
 'Vada (1)',
 'Akki Rotti (2)',
 'Gheee Plain Dosa',
 'Onion Dosa',
 'Open Butter Masala',
 'Healthy Ragi Rotti / Nachni Roti (2)',
 'Rava Dosa',
 'Rava Masala Dosa',
 'Rava Onion Dosa',
 'Cream Of Veg Soup',
 'Hot & Sour Soup',
 'Sweet Corn Veg Soup',
 'Tomato Soup',
 'Veg Clear Soup',
 'Veg Manchow Soup',
 'Babycorn Chilli',
 'Babycorn Pepper Dry',
 'Corn Kabab',
 'Crispy Babycorn',
 'Finger Chips',
 'Gobi Chilli',
 'Gobi Pepper Dry',
 'Harabhara Kabab',
 'Mushroom Chilli',
 'Mushroom Manchurian',
 'Mushroom Pepper Dry',
 'Paneer Chilli',
 'Paneer Manchurian',
 'Pan

In [31]:
def recommend_dish(rest_id, selected_dish):
    # Filter dataset based on rest_id and selected dish
    selected_dish_data = filtered_df[(filtered_df['Rest_id'] == rest_id) & (filtered_df['menu'] == selected_dish)]
    if selected_dish_data.empty:
        return "Selected dish not found for the given rest_id."

    # Extract menu type, dish votes, and dish rating of selected dish
    menu_type = selected_dish_data['Menu Type'].iloc[0]
    dish_votes = selected_dish_data['DishVotes'].iloc[0]
    dish_rating = selected_dish_data['DishRating'].iloc[0]

    # Filter dataset based on rest_id, menu type, dish votes, and dish rating
    filtered_data = filtered_df[(filtered_df['Rest_id'] == rest_id) &
                                (filtered_df['Menu Type'] == menu_type) &
                                (filtered_df['DishVotes'] == dish_votes) &
                                (filtered_df['DishRating'] == dish_rating)]

    # Exclude selected dish from filtered dataset
    filtered_data = filtered_data[filtered_data['menu'] != selected_dish]

    # Sort the filtered dataset based on some similarity score
    filtered_data['SimilarityScore'] = filtered_data['DishVotes'] + filtered_data['DishRating']
    filtered_data = filtered_data.sort_values(by='SimilarityScore', ascending=False)

    # Retrieve recommended dishes
    recommended_dishes = filtered_data['menu'].tolist()[:10]

    return recommended_dishes


In [32]:

recommend_dish(84508,'Hot Chocolate Fudge')

['Mango Fruit Milkshake',
 'Strawberry Ice Cream Milkshake',
 'Mosambi Juice',
 'Musk Melon Juice',
 'Pineapple Juice',
 'Salt Lassi',
 'Sweet Lassi',
 'Apple Milkshake',
 'Mango Fruit Milkshake',
 'Black Current Ice Cream Milkshake']

## Old Version

In [93]:
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommendation_model():
    # Step 1: Select Restaurant
    rest_name = input("Please enter the name of the restaurant: ")

    # Filter the dataset based on the selected restaurant name
    rest_data = filtered_df[filtered_df['Rest_Name'] == rest_name]
    #print(rest_data)
    if rest_data.empty:
        return "Restaurant not found. Please try again."

    rest_id = rest_data['Rest_id'].iloc[0]

    # Step 2: Select Menu Type
    unique_menu_types = rest_data['Menu Type'].unique()
    print("Available Menu Types:")
    for i, menu_type in enumerate(unique_menu_types, start=1):
        print(f"{i}. {menu_type}")

    #menu_type_choice = input("Please select a menu type: ")
    #menu_type = unique_menu_types[int(menu_type_choice) - 1]
    menu_type_choice = input("Please select a menu type (veg/nonveg): ")
    menu_type = menu_type_choice.lower().strip()

    # Filter the dataset based on the selected restaurant and menu type
    menu_type_data = rest_data[rest_data['Menu Type'] == menu_type]
    if menu_type_data.empty:
        return "No menu found for the selected type. Please try again."

    # Step 3: Select BestSeller or Not
    bestseller_choice = input("Do you want bestseller dishes only? (yes/no): ")
    if bestseller_choice.lower() == 'yes':
        filtered_data = menu_type_data[menu_type_data['BestSeller'] == 'BestSeller']
    else:
        filtered_data = menu_type_data

    # Display the unique menu options that satisfy the selected conditions
    available_menu = filtered_data['menu'].unique()
    print("Available Menu Options:")
    for i, menu in enumerate(available_menu, start=1):
        print(f"{i}. {menu}")

    # Step 4: Select Menu Options
    menu_choices = input("Select the menu options (comma-separated menu names): ")
    menu_choices = [choice.strip() for choice in menu_choices.split(',')]

    selected_menus = [menu for menu in menu_choices if menu in available_menu]

    # Step 5: Ask for Recommendation
    recommend_choice = input("Would you like to receive recommendations? (yes/no): ")
    if recommend_choice.lower() == 'yes':
        # Filter the dataset based on selected menu options
        #print(selected_menus)
        selected_data = filtered_data[filtered_data['menu'].isin(selected_menus)]
        #print(selected_data)
        # Extract the numeric value from DishVotes
        selected_data['DishVotes'] = selected_data['DishVotes'].str.extract('(\d+)').astype(float)
        menu_type_data['DishVotes'] = menu_type_data['DishVotes'].str.extract('(\d+)').astype(float)
        # Calculate the similarity score based on DishVotes and DishRating
        features = menu_type_data[['DishVotes', 'DishRating']].fillna(0)
        similarity_matrix = cosine_similarity(features)
        #print (similarity_matrix)
        #print(similarity_matrix.shape[0])
        # Retrieve the recommended dishes
        recommended_dishes = []
        for i in range(similarity_matrix.shape[0]):
            similar_indices = np.argsort(similarity_matrix[i])[::-1]  # Sort in descending order of similarity
            #print(similar_indices)
            #similar_dishes = menu_type_data.iloc[similar_indices]['menu'].tolist()
            similar_dishes = list(set(menu_type_data.iloc[similar_indices]['menu'].tolist()))
            for dish in similar_dishes:
                if dish not in selected_menus:
                    recommended_dishes.append(dish)
                    if len(recommended_dishes) == 10:
                        break
            if len(recommended_dishes) == 10:
                break
    
        recommended_dishes.sort(key=lambda x: x[1], reverse=True)

        return recommended_dishes

        return recommended_dishes

    return "Recommendation not requested. Thank you!"


# Example usage of the recommendation model
recommendations = recommendation_model()
print(recommendations)


Please enter the name of the restaurant: Anna Kuteera
Available Menu Types:
1. veg
Please select a menu type (veg/nonveg): veg
Do you want bestseller dishes only? (yes/no): no
Available Menu Options:
1. Veg Grill Cheese Sandwich
2. Masala Puri
3. Rava Idly (1 Pc)
4. Pure Ghee Masala Dosa
5. Veg Noodles
6. Gobi Manchurian
7. Mushroom Fried Rice
8. Veg Fried Rice
9. Jeera Rice
10. Bhel Puri
11. Crispy Babycorn
12. Pav Bhaji
13. Set Dosa (2)
14. Babycorn Manchurian
15. Idly (2 Pc)
16. Dahi Puri
17. Paneer Manchurian
18. Onion Dosa
19. Open Butter Masala
20. Corn Kabab
21. Harabhara Kabab
22. Sev Puri
23. Mango Fruit Milkshake
24. Pani Puri
25. Samosa Masala
26. Jamoon
27. Tomato Soup
28. Coffee
29. Veg Manchow Soup
30. Carrot Halwa
31. Buns(1pcs)
32. Masala Vada(2)
33. 1 Idly 1 Vada
34. 2 Idly 1 Vada
35. Vada (1)
36. Akki Rotti (2)
37. Gheee Plain Dosa
38. Healthy Ragi Rotti / Nachni Roti (2)
39. Rava Dosa
40. Rava Masala Dosa
41. Rava Onion Dosa
42. Cream Of Veg Soup
43. Hot & Sour Soup


C:\Users\SNIGGDHA\AppData\Local\Temp/ipykernel_2324/3351118310.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['DishVotes'] = selected_data['DishVotes'].str.extract('(\d+)').astype(float)
